In [7]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

# dataset = pd.read_csv('./data/dataset_clean.csv', index_col=0)


In [8]:
# dataset

In [9]:
data = pd.read_csv('C:\\Users\\Utilisateur\\Desktop\\project_dev\\price_project\\appli_copy\\data\\train.csv')

In [10]:
Correct_Mileage= []
for i in data.Mileage:
    if str(i).endswith('km/kg'):
        i = i[:-6]
        i = float(i)*1.40
        Correct_Mileage.append(float(i))
    elif str(i).endswith('kmpl'):
        i = i[:-6]
        #print(i)
        Correct_Mileage.append(float(i))
    else :
        Correct_Mileage.append(float(i))


data['Mileage']=Correct_Mileage

# dataset_brand = pd.read_csv('C:\\Users\\Utilisateur\\Desktop\\project_dev\\price_project\\brand.csv')['Brand']

# names = data['Name']
# final_brands = []
# final_names = []

# for name in names :
#     find = False
#     name = name
#     for brand in dataset_brand :        
#         if name.find(brand) != -1 :
#             final_brands.append(brand)
#             try :
#                 final_names.append(name.split(brand + ' ')[1])
#             except :
#                 final_names.append(name.split(brand + ' ')[0])
#             find = True
#     if find == False :
#         final_brands.append(name.split(' ')[0])
#         final_names.append(name.split(' ')[1])
        
# print(final_names)
        
# data['Brand'] = final_brands

# data = data.drop(['Name', 'Location'], axis=1)


data['Engine'] = data['Engine'].str.strip(' CC')
data['Power'] = data['Power'].str.strip(' bph')
data['Power'] = data['Power'].str.replace('null', '0')
data['New_Price'] = data['New_Price'].str.strip(' Lakh')
data['New_Price'] = data['New_Price'].str.strip(' Cr')
data['New_Price'] = data['New_Price'].astype(float).fillna(0)
data = data.fillna(0)

In [11]:
print('Land Rover Range Rover 2.2L Pure'.find('Land Rover'))

0


In [12]:
data['Price']       = round(((data['Price'] * 1219.74)), 2)
data['New_Price']   = round(((data['New_Price'].astype(float) * 1219.74)), 2)


data.to_csv('C:\\Users\\Utilisateur\\Desktop\\project_dev\\price_project\\api\\data\\dataset.csv', index=False)


In [13]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

string_features         = ['Name', 'Location']
categorical_features    = ['Fuel_Type', 'Transmission', 'Owner_Type']
numeric_features        = ['Year', 'Kilometers_Driven', 'Mileage', 'Engine', 'Power', 'Seats', 'New_Price']

string_transformer = Pipeline(steps=[
    ('label', OrdinalEncoder()),
    ('scaler', StandardScaler())
])

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

In [14]:
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('str', string_transformer, string_features),
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


In [15]:
preprocessor

ColumnTransformer(transformers=[('str',
                                 Pipeline(steps=[('label', OrdinalEncoder()),
                                                 ('scaler', StandardScaler())]),
                                 ['Name', 'Location']),
                                ('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['Year', 'Kilometers_Driven', 'Mileage',
                                  'Engine', 'Power', 'Seats', 'New_Price']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 ['Fuel_Type', 'Transmission', 'Owner_Type'])])

In [16]:
y_data = data['Price']
# features 
X_data = data.drop('Price',axis=1)

In [17]:
svr_rbf_all = SVR(kernel='rbf', C=100, degree=2, gamma=0.01)


sts = StandardScaler()
y_data = sts.fit_transform(pd.DataFrame(y_data))


# y_data = sts.inverse_transform(y_data)
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.2, random_state=42)

X_data_all = preprocessor.fit_transform(X_data)

print(pd.DataFrame(X_data_all).head())
# pd.DataFrame(X_data_all, columns=X_data.columns).to_csv('X_data_all.csv', index=False)
# print(X_data_all)
svr_rbf_all.fit(X_data_all, y_data)
# X_test = preprocessor.fit_transform(X_test)

y_pred_all = svr_rbf_all.predict(X_data_all)

print(r2_score(y_data, y_pred_all))

print(mean_squared_error(y_data, y_pred_all))

c:\Users\Utilisateur\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\Utilisateur\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


         0         1         2         3         4         5         6   \
0  0.536130  1.143658 -1.027139  0.145315  3.930940 -1.001928 -0.936570   
1 -0.762552  1.481989  0.502161 -0.194369  0.329818 -0.048301  0.279486   
2 -0.811630 -1.224657 -0.721279 -0.139581  0.003186 -0.673710 -0.390739   
3  0.269976 -1.224657 -0.415419  0.309678  0.554378 -0.593697 -0.389666   
4 -1.685597 -0.886327 -0.109559 -0.197985 -0.609250  0.582007  0.540427   

         7         8    9    10   11   12   13   14   15   16   17   18   19  
0 -0.263523 -0.272039  1.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  
1 -0.263523 -0.272039  0.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  
2 -0.263523  0.569922  0.0  0.0  0.0  0.0  1.0  0.0  1.0  1.0  0.0  0.0  0.0  
3  1.915247 -0.272039  0.0  1.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  
4 -0.263523 -0.272039  0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  
0.9018810821404337
0.09811891785956622


In [18]:
numeric_features_lite        = ['Year', 'Engine', 'Power', 'New_Price']

numeric_transformer_lite = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])


from sklearn.compose import ColumnTransformer
preprocessor_lite = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_lite, numeric_features_lite),
    ])

svr_rbf_lite = SVR(kernel='rbf', C=100, degree=2, gamma=0.01)

X_data_lite = X_data.loc[:, numeric_features_lite]

In [19]:

# print(preprocessor_lite.transform(pd.DataFrame([[2010, 998, 58.16, 0]], columns=numeric_features_lite)))



# import pickle

# with open('C:\\Users\\Utilisateur\\Desktop\\project_dev\\price_project\\api\\data\\all\\model.pkl', 'wb') as file :
#     pickle.dump(svr_rbf_all, file)
    
# file.close()

# with open('C:\\Users\\Utilisateur\\Desktop\\project_dev\\price_project\\api\\data\\all\\preprocessor.pkl', 'wb') as file :
#     pickle.dump(preprocessor, file)
    
# file.close()

In [20]:
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split


# X_train_lite, X_test_lite, y_train_lite, y_test_lite = train_test_split(X_data_lite, y_data, test_size = 0.2, random_state=42)

X_data_lite = preprocessor_lite.fit_transform(X_data_lite)

svr_rbf_lite.fit(X_data_lite, y_data)

# X_train_lite = preprocessor_lite.fit_transform(X_train_lite)

# svr_rbf_lite.fit(X_train_lite, y_train_lite)

# X_test_lite = preprocessor_lite.transform(X_test_lite)

y_pred_lite = svr_rbf_lite.predict(X_data_lite)

print(r2_score(y_data, y_pred_lite))

print(mean_squared_error(y_data, y_pred_lite))

c:\Users\Utilisateur\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8385958024030764
0.16140419759692354


In [21]:

# import pickle

# with open('C:\\Users\\Utilisateur\\Desktop\\project_dev\\price_project\\api\\data\\lite\\model.pkl', 'wb') as file :
#     pickle.dump(svr_rbf_lite, file)
    
# file.close()

# with open('C:\\Users\\Utilisateur\\Desktop\\project_dev\\price_project\\api\\data\\lite\\preprocessor.pkl', 'wb') as file :
#     pickle.dump(preprocessor_lite, file)
    
# file.close()

In [22]:
# svr_rbf.fit(X_data, y_data)

In [23]:
# X_test = preprocessor.fit_transform(X_test)
# y_pred = svr_rbf.predict(X_data)

In [24]:
import pickle

# with open('C:\\Users\\Utilisateur\\Desktop\\project_dev\\price_project\\api\\data\\model_final.pkl', 'wb') as file :
#     pickle.dump(svr_rbf, file)
    
# file.close()

# with open('C:\\Users\\Utilisateur\\Desktop\\project_dev\\price_project\\api\\data\\preprocessor.pkl', 'wb') as file :
#     pickle.dump(preprocessor, file)
    
# file.close()

In [25]:
# with open('C:\\Users\\Utilisateur\\Desktop\\project_dev\\price_project\\api\\data\\com\\target.pkl', 'wb') as file :
#     pickle.dump(sts, file)
    
# file.close()